# **Planificación de Horarios para Trabajadores en una Planta de Producción**

Una planta de producción tiene un número de trabajadores, cada uno con un conjunto de habilidades específicas y horas de disponibilidad. La planta tiene diferentes turnos a cubrir, cada uno con requisitos específicos de habilidades y una duración en horas.

El objetivo es planificar los horarios de trabajo de manera que se cumplan los requisitos de cada turno, minimizando las horas extra trabajadas y maximizando la cantidad de turnos cubiertos.

**Los datos incluyen:**

* Una lista de trabajadores, cada uno con:
* Sus habilidades (por ejemplo, soldadura, ensamblaje, control de calidad, etc.).
* Su disponibilidad en horas (por ejemplo, 40 horas semanales).
* Una lista de turnos, cada uno con:
    * Requisitos de habilidades (por ejemplo, se necesita un trabajador que sepa soldar y otro para ensamblaje).
    * Duración en horas.
    * Prioridad (algunos turnos son más importantes que otros).

Escribir un programa en Python que, usando programación dinámica, asigne turnos a los trabajadores de manera óptima, maximizando la cantidad de turnos cubiertos y minimizando las horas extras.


In [1]:
from collections import defaultdict

class Worker:
    def __init__(self, name, skills, availability):
        self.name = name
        self.skills = set(skills)
        self.availability = availability
        self.assigned_hours = 0

class Shift:
    def __init__(self, skills_required, duration, priority):
        self.skills_required = skills_required
        self.duration = duration
        self.priority = priority

def assign_shifts(workers, shifts):
    # Sort shifts by priority (higher priority first)
    shifts.sort(key=lambda x: x.priority, reverse=True)
    
    # DP table to store the best assignment
    dp = defaultdict(lambda: float('-inf'))
    dp[0] = 0  # Base case: no shifts assigned, no hours worked
    
    # Track the best assignment
    best_assignment = {}
    
    for shift in shifts:
        new_dp = dp.copy()
        for state, value in dp.items():
            for worker in workers:
                if worker.skills >= set(shift.skills_required) and worker.availability - worker.assigned_hours >= shift.duration:
                    new_state = state | (1 << shifts.index(shift))
                    new_value = value + shift.priority
                    if new_value > new_dp[new_state]:
                        new_dp[new_state] = new_value
                        best_assignment[new_state] = (worker, shift)
                        worker.assigned_hours += shift.duration
        dp = new_dp
    
    # Extract the best assignment
    max_state = max(dp, key=dp.get)
    assignment = []
    while max_state:
        worker, shift = best_assignment[max_state]
        assignment.append((worker.name, shift.skills_required, shift.duration))
        max_state &= ~(1 << shifts.index(shift))
    
    return assignment

# Example usage
workers = [
    Worker("Alice", ["welding", "assembly"], 40),
    Worker("Bob", ["quality_control", "assembly"], 40),
    Worker("Charlie", ["welding", "quality_control"], 40)
]

shifts = [
    Shift(["welding", "assembly"], 8, 3),
    Shift(["quality_control"], 4, 2),
    Shift(["assembly"], 6, 1)
]

assignment = assign_shifts(workers, shifts)
for a in assignment:
    print(f"Worker {a[0]} assigned to shift requiring {a[1]} for {a[2]} hours")

Worker Alice assigned to shift requiring ['assembly'] for 6 hours
Worker Bob assigned to shift requiring ['quality_control'] for 4 hours
Worker Alice assigned to shift requiring ['welding', 'assembly'] for 8 hours


In [3]:
from typing import List, Set, Dict
from dataclasses import dataclass

@dataclass
class Worker:
    id: int
    skills: Set[str]
    available_hours: int
    assigned_hours: int = 0

@dataclass
class Shift:
    id: int
    required_skills: Set[str]
    duration: int
    priority: int

class ShiftScheduler:
    def __init__(self, workers: List[Worker], shifts: List[Shift]):
        self.workers = workers
        self.shifts = shifts
        self.memo = {}

    def can_worker_cover_shift(self, worker: Worker, shift: Shift) -> bool:
        return (shift.required_skills.issubset(worker.skills) and 
                worker.available_hours - worker.assigned_hours >= shift.duration)

    def find_optimal_schedule(self) -> Dict[int, List[int]]:
        # Key: shift_id, Value: List of worker_ids
        schedule = {}
        
        # Sort shifts by priority (descending)
        sorted_shifts = sorted(self.shifts, key=lambda x: x.priority, reverse=True)
        
        for shift in sorted_shifts:
            best_workers = self._find_best_workers_for_shift(shift)
            if best_workers:
                schedule[shift.id] = [w.id for w in best_workers]
                for worker in best_workers:
                    worker.assigned_hours += shift.duration
        
        return schedule

    def _find_best_workers_for_shift(self, shift: Shift) -> List[Worker]:
        available_workers = [
            w for w in self.workers 
            if self.can_worker_cover_shift(w, shift)
        ]
        
        # Find workers with minimum assigned hours that can cover the shift
        if available_workers:
            available_workers.sort(key=lambda w: w.assigned_hours)
            return [available_workers[0]]
        
        return []

# Example usage
def main():
    # Create workers
    workers = [
        Worker(1, {"soldadura", "ensamblaje"}, 40),
        Worker(2, {"ensamblaje", "control_calidad"}, 40),
        Worker(3, {"soldadura", "control_calidad"}, 40),
    ]

    # Create shifts
    shifts = [
        Shift(1, {"soldadura", "ensamblaje"}, 8, 3),
        Shift(2, {"ensamblaje", "control_calidad"}, 8, 2),
        Shift(3, {"soldadura", "control_calidad"}, 8, 1),
    ]

    # Create scheduler and find optimal schedule
    scheduler = ShiftScheduler(workers, shifts)
    schedule = scheduler.find_optimal_schedule()

    # Print results
    print("Optimal Schedule:")
    for shift_id, worker_ids in schedule.items():
        print(f"Shift {shift_id}: Workers {worker_ids}")
    
    print("\nWorker Hours:")
    for worker in workers:
        print(f"Worker {worker.id}: {worker.assigned_hours}/{worker.available_hours} hours")

if __name__ == "__main__":
    main()

Optimal Schedule:
Shift 1: Workers [1]
Shift 2: Workers [2]
Shift 3: Workers [3]

Worker Hours:
Worker 1: 8/40 hours
Worker 2: 8/40 hours
Worker 3: 8/40 hours
